<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.6: Generators: Types
**Prev: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**<br>
**Next: [Introduction to FIRRTL](4.1_firrtl_ast.ipynb)**

## Motivation
Scala is a strongly-typed programming language.
This is a two-edged sword; on one hand, many programs that would compile and execute in Python (a dynamically-typed language) would fail at compile time in Scala.
On the other hand, programs that compile in Scala will contain many fewer runtime errors than a similar Python program.

In this section, our goal is to familiarize you with types as a first class citizen in Scala.
While initially you may feel you have limited productivity, you will soon learn to understand compile-time error messages and how to architect your programs with the type system in mind to catch more errors for you. 


## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "e:\\Academics\\PhD\\4.Chisel\\chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Static Types<a name="types-in-scala"></a>

## Types in Scala

All objects in Scala have a type, which is usually the object's class.
Let's see some:

In [3]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)     // this is a java string because Scala is built on top of Java

int
double
class java.lang.String


When you declare your own class, it has an associated type.

In [4]:
class MyClass {
    def myMethod = ???
}

// this has an associated type
println(new MyClass().getClass)

// The type ammonite.$sess.cmd6$Helper$MyClass refers to a class that was defined within an Ammonite REPL session.

// ammonite: This is the package used by the Ammonite Scala REPL, an enhanced Scala shell.

// $sess: Indicates this is part of the interactive session.

// cmd6: The class was defined in the 6th input command during the session.

// Helper: Ammonite sometimes wraps session code in a helper object for evaluation.

// MyClass: The actual class you defined.

class ammonite.$sess.cmd3$Helper$MyClass


defined class MyClass

While not required, it is HIGHLY recommended that you **define input and output types for all function declarations**.
This will let the Scala compiler catch improper use of a function.

In [5]:
def double(s: String): String = s + s
// Uncomment the code below to test it
double("hi")      // Proper use of double
// double(10)        // Bad input argument!, type mismatch will occur
// double("hi") / 10 // Inproper use of double's output!, output is not a double but a String

defined function double
res4_1: String = "hihi"

Functions that don't return anything return type `Unit`.

In [6]:
// create the counter variable
var counter = 0

// call the method to increment the variable
// since it is global, the function can access it
def increment(): Unit = {
    counter += 1
}

// then call the function
increment()

counter: Int = 1
defined function increment

## Scala vs. Chisel Types<a name="scala-vs-chisel-types"></a>

Recap: Module 2.2 discussed the difference between Chisel types and Scala types, for example the fact that
```scala
val a = Wire(UInt(4.W))
a := 0.U
```
is legal because `0.U` is of type `UInt` (a Chisel type), whereas
```scala
val a = Wire(UInt(4.W))
a := 0
```
is illegal because 0 is type `Int` (a Scala type).

This is also true of `Bool`, a Chisel type which is distinct from `Boolean`.
```scala
val bool = Wire(Bool())
val boolean: Boolean = false
// legal
when (bool) { ... }
if (boolean) { ... }
// illegal
if (bool) { ... }
when (boolean) { ... }
```

If you make a mistake and mix up `UInt` and `Int` or `Bool` and `Boolean`, the Scala compiler will generally catch it for you.
This is because of Scala's static typing.
At compile time, the compiler is able to distinguish between Chisel and Scala types and also able to understand that `if ()` expects a `Boolean` and `when ()` expects a `Bool`.


## Scala Type Coercion (Automatically converting a value from one type to another )<a name="type-coercion"></a>

<!-- typeOf. Scala has a function called `typeOf[T]` which returns a type object for `T`. -->
<!-- This doesn't actually seem useful to Chisel users... -->

### asInstanceOf

`x.asInstanceOf[T]` casts the object `x` to the type `T`. It throws an exception if the given object cannot be cast to type `T`.

In [7]:
// create the chisel type variable x
val x: UInt = 3.U

try {
  // try to convert x to an Int
  println(x.asInstanceOf[Int])
} catch {
  // UInt cant be coverted to Int
  // this error will be thrown
  case e: java.lang.ClassCastException => println("As expected, we can't cast UInt to Int")
}

// But we can cast UInt to Data since UInt inherits from chisel3.Data, therefore casting is possible
println(x.asInstanceOf[chisel3.Data])

As expected, we can't cast UInt to Int
UInt<2>(3)


x: UInt = UInt<2>(3)


### Type Casting in Chisel

The code below will give an error if you try to run it without removing the comment.
What's the problem?
It is trying to assign a `UInt` to an `SInt`, which is illegal.

Chisel has a set of type casting functions.
The most general is `asTypeOf()`, which is shown below.
Some chisel objects also define `asUInt()` and `asSInt()` as well as some others.

If you remove the `//` from the code block below, the example should work for you.


In [8]:
class TypeConvertDemo extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(SInt(4.W))
    })

    // this is not possible
    // chisel does not allow to assign a UInt to an SInt
    // this will be in a infinite running sceneario
    // io.out := io.in 

    // this is okay since type casting happens
    io.out := io.in.asTypeOf(io.out) // also for typecasting we can use asUInt() , asSInt() , etc
}

test(new TypeConvertDemo) { c =>
      c.io.in.poke(3.U)
      c.io.out.expect(3.S)
      c.io.in.poke(15.U)
      c.io.out.expect(-1.S)
}

Elaborating design...
Done elaborating.
test TypeConvertDemo Success: 0 tests passed in 2 cycles in 0.028309 seconds 70.65 Hz


defined class TypeConvertDemo

---
# Type Matching<a name="type-matching"></a>

## Match Operator
Recall that in 3.1 the match operator was introduced.
Type matching is especially useful when trying to write type-generic generators.
The following example shows an example of a "generator" that can add two literals of type `UInt` or `SInt`.
Later sections will talk more about writing type-generic generators.

**Note: there are much better and safer ways to write type-generic generators in Scala**.

In [9]:
class ConstantSum(in1: chisel3.Data, in2: chisel3.Data) extends Module {
    val io = IO(new Bundle {

        // we can dynamically set the type of output to be the output of the input
        val out = Output(chiselTypeOf(in1)) // in case in1 is literal then just get its type
    })
    (in1, in2) match {

        // if both are unsigned integers
        case (x: UInt, y: UInt) => io.out := x + y

        // if both are signed integers
        case (x: SInt, y: SInt) => io.out := x + y

        // any other type combinations
        case _ => throw new Exception("I give up!") // if we pass any other value, an error will be thrown
    }
}
println(getVerilog(dut = new ConstantSum(3.U, 4.U)))    // this is acceptable
println(getVerilog(dut = new ConstantSum(-3.S, 4.S)))   // this is acceptable
// println(getVerilog(dut = new ConstantSum(3.U, 4.S)))    // an error will be thrown


Elaborating design...
Done elaborating.
module ConstantSum(
  input        clock,
  input        reset,
  output [1:0] io_out
);
  assign io_out = 2'h3; // @[cmd8.sc 10:43]
endmodule

Elaborating design...
Done elaborating.
module ConstantSum(
  input        clock,
  input        reset,
  output [2:0] io_out
);
  assign io_out = 3'sh1; // @[cmd8.sc 13:43]
endmodule



defined class ConstantSum

It is good to remember that Chisel types generally should not be value matched.
Scala's match executes during circuit elaboration, but what you probably want is a post-elaboration comparison.
The following gives a syntax error:

In [9]:
// this will give a syntax error

class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })

    // this is not accurate code
    io.out := (io.in match {
        // note that case 0.U is an error
        // because it is matched to the value
        // Chisel thypes generally cant be value matched
        case (0.U) => true.B
        case _   => false.B
    })

    // accurate code
    // when(io.in === 0.U){
    //     io.out := true.B
    // }.otherwise{
    //     io.out := false.B
    // }
}
println(getVerilog(new InputIsZero))

(console):14:16 expected ")"
// this will give a syntax error

class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })

    // this is not accurate code
    io.out := (io.in match {
        // note that case 0.U is an error
        // because it is matched to the value
        // Chisel thypes generally cant be value matched
        case (0.U) => true.B
        case _   => false.B
    })

    // accurate code
    // when(io.in === 0.U){
    //     io.out := true.B
    // }.otherwise{
    //     io.out := false.B
    // }
}
println(getVerilog(new InputIsZero))
                                                                                                                                                                                                                                                                                                                                                              

: 

## Unapply
What's actually going on when you do a match?
How does Scala let you do fancy value matching with case classes like this:
```scala
case class Something(a: String, b: Int)
val a = Something("A", 3)
a match {
    case Something("A", value) => value
    case Something(str, 3)     => 0
}
```

As it turns out, the companion object that is created for every case class also contains an **unapply** method, in addition to an **apply** method.
What is an **unapply** method?

Scala unapply methods are another form of syntactic sugar that give match statements the ability to both match on types and **extract values** from those types during the matching.

Let's look at the following example.
For some reason, let's say that if the generator is being pipelined, the delay is `3*totalWidth`, otherwise the delay is `2*someOtherWidth`.
Because case classes have **unapply** defined, we can match values inside the case class, like so:

In [ ]:
// first create a case class
case class SomeGeneratorParameters(
    // case class has three parameters
    someWidth: Int,

    // these two parameters can be option, otherwise a default value will be used
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {

    // check for parameter requirements
    require(someWidth >= 0)
    require(someOtherWidth >= 0)

    // calculate the totalWidth
    val totalWidth = someWidth + someOtherWidth
}

// now create a function, which accepts SomeGeneratorParameters object
def delay(p: SomeGeneratorParameters): Int = p match {

    // match the false case for pipelineMe argument
    // if that case matches, return 2 * someOtherWidth
    case SomeGeneratorParameters(_, sw, false) => sw * 2

    // match the true case for pipelineMe argument for true case
    // here assign this matching object to sg
    // if the case matches, return totalWidth * 3 as the output
    case sg @SomeGeneratorParameters(_, _, true) => sg.totalWidth * 3
}

// here pipelineMe is false
// therefore 10 * 2 will be printed
println(delay(SomeGeneratorParameters(10, 10)))

// here pipelineMe is true
// the 20 * 3 will be printed
println(delay(SomeGeneratorParameters(10, 10, true)))

20
60


defined class SomeGeneratorParameters
defined function delay

If you look at the `delay` function, you should note that in addition to matching on the type of each character, we are also:
- Directly referencing internal values of the parameters
- Sometimes, are matching directly on the internal values of the parameters

These are possible due to the compiler implementing an `unapply` method. Note that unapplying the case is just syntactic sugar; e.g. the following two cases examples are equivalent:
```scala
case p: SomeGeneratorParameters => p.sw * 2
case SomeGeneratorParameters(_, sw, _) => sw * 2
```

In addition, there are more syntaxes and styles of matching. The following two cases are also equivalent, but the second allows you to match on internal values while still referencing the parent value:
```scala
case SomeGeneratorParameters(_, sw, true) => sw
case sg @SomeGeneratorParameters(_, sw, true) => sw
```

Finally, you can directly embed condition checking into match statements, as demonstrated by the third of these equivalent examples:
```scala
case SomeGeneratorParameters(_, sw, false) => sw * 2
case s @SomeGeneratorParameters(_, sw, false) => s.sw * 2
case s: SomeGeneratorParameters if s.pipelineMe => s.sw * 2
```

All these syntaxes are enabled by a Scala unapply method contained in a class's companion object. If you want to unapply a class but do not want to make it a case class, you can manually implement the unapply method. The following example demonstrates how one can manually implement a class's apply and unapply methods:

In [ ]:
// when you create a case class , unapply method will be automatically created

// when you create normal classes, unapply method has to be manually created

// create the Boat Class and Companion object
class Boat(val name: String, val length: Int)

// Companion Object
object Boat {

    // create the unapply method, this will return an Option which is a Some
    // it contains name and the length of the boat
    def unapply(b: Boat): Option[(String, Int)] = Some((b.name, b.length))

    // apply method will just create a new Boat object, which is the usual behavior
    def apply(name: String, length: Int): Boat = new Boat(name, length)
}

// create the function which does the matching
// it requires a list of boats and will return a list of boats after filtering
def getSmallBoats(seq: Seq[Boat]): Seq[Boat] = seq.filter { b =>

    b match {

        // check if the length is lower than 60, if so return true
        case Boat(_, length) if length < 60 => true

        // otherwise return false
        case Boat(_, _) => false
    }
}

// create a list of boats, amoung them last two boats will return true when the 
// function is applied
val boats = Seq(Boat("Santa Maria", 62), Boat("Pinta", 56), Boat("Nina", 50))
println(getSmallBoats(boats).map(_.name).mkString(" and ") + " are small boats!")

// Key point - Unapply method helps with matching , not with the types but also with the internal parameters of an object

Pinta and Nina are small boats!


defined class Boat
defined object Boat
defined function getSmallBoats
boats: Seq[Boat] = List(
  ammonite.$sess.cmd18$Helper$Boat@47e313ee,
  ammonite.$sess.cmd18$Helper$Boat@64c790ea,
  ammonite.$sess.cmd18$Helper$Boat@57be57b6
)

## Partial Functions
This is a brief overview; [this guide](https://twitter.github.io/scala_school/pattern-matching-and-functional-composition.html#PartialFunction) has a more detailed overview.

Partial functions are functions that are only defined on a subset of their inputs.
Like an option, a partial function may not have a value for a particular input.
This can be tested with `isDefinedAt(...)`.

Partial functions can be chained together with `orElse`.

Note that calling a `PartialFunction` with an undefined input will result in a runtime error. This can happen, for example, if the input to the `PartialFunction` is user-defined. To be more type-safe, we recommend writing functions that return an `Option` instead.

In [ ]:
// Helper function to make this cell a bit less tedious.
def printAndAssert(cmd: String, result: Boolean, expected: Boolean): Unit = {
  println(s"$cmd = $result")
  assert(result == expected)
}

// Defined for -1, 2, 5, etc.
val partialFunc1: PartialFunction[Int, String] = {
  case i if (i + 1) % 3 == 0 => "Something"

  // for other integers, there are no output
}

// check for isDefinedAt using that function with the help of the helper function
printAndAssert("partialFunc1.isDefinedAt(2)", partialFunc1.isDefinedAt(2), true)  // it is defined for 2
printAndAssert("partialFunc1.isDefinedAt(5)", partialFunc1.isDefinedAt(5), true)  // it is defined for 5
printAndAssert("partialFunc1.isDefinedAt(1)", partialFunc1.isDefinedAt(1), false) // it is not defined for 1
printAndAssert("partialFunc1.isDefinedAt(0)", partialFunc1.isDefinedAt(0), false) // it is not defined for 0
println(s"partialFunc1(2) = ${partialFunc1(2)}")  // print Something if defined

// it is not defined for 0, therefore catch block will catch the error
try {
  println(partialFunc1(0))
} catch {
  case e: scala.MatchError => println("partialFunc1(0) = can't apply PartialFunctions where they are not defined")
}

// Defined for 1, 4, 7, etc.
val partialFunc2: PartialFunction[Int, String] = {
  case i if (i + 2) % 3 == 0 => "Something else"
}
printAndAssert("partialFunc2.isDefinedAt(1)", partialFunc2.isDefinedAt(1), true)
printAndAssert("partialFunc2.isDefinedAt(0)", partialFunc2.isDefinedAt(0), false)
println(s"partialFunc2(1) = ${partialFunc2(1)}")
try {
  println(partialFunc2(0))
} catch {
  case e: scala.MatchError => println("partialFunc2(0) = can't apply PartialFunctions where they are not defined")
}

// this partial function is chained use orElse
val partialFunc3 = partialFunc1 orElse partialFunc2
printAndAssert("partialFunc3.isDefinedAt(0)", partialFunc3.isDefinedAt(0), false)
printAndAssert("partialFunc3.isDefinedAt(1)", partialFunc3.isDefinedAt(1), true)
printAndAssert("partialFunc3.isDefinedAt(2)", partialFunc3.isDefinedAt(2), true)
printAndAssert("partialFunc3.isDefinedAt(3)", partialFunc3.isDefinedAt(3), false)
println(s"partialFunc3(1) = ${partialFunc3(1)}")
println(s"partialFunc3(2) = ${partialFunc3(2)}")

partialFunc1.isDefinedAt(2) = true
partialFunc1.isDefinedAt(5) = true
partialFunc1.isDefinedAt(1) = false
partialFunc1.isDefinedAt(0) = false
partialFunc1(2) = Something
partialFunc1(0) = can't apply PartialFunctions where they are not defined
partialFunc2.isDefinedAt(1) = true
partialFunc2.isDefinedAt(0) = false
partialFunc2(1) = Something else
partialFunc2(0) = can't apply PartialFunctions where they are not defined
partialFunc3.isDefinedAt(0) = false
partialFunc3.isDefinedAt(1) = true
partialFunc3.isDefinedAt(2) = true
partialFunc3.isDefinedAt(3) = false
partialFunc3(1) = Something else
partialFunc3(2) = Something


defined function printAndAssert
partialFunc1: PartialFunction[Int, String] = <function1>
partialFunc2: PartialFunction[Int, String] = <function1>
partialFunc3: PartialFunction[Int, String] = <function1>

---
# Type Safe Connections<a name="type-safe-connections"></a>

Chisel can check the type for many connections, including:
* Bool/UInt to Clock

For other types, Chisel will let you connect them, but may truncate/pad bits as appropriate.
* Bool/UInt to Bool/UInt
* Bundle to Bundle

In [ ]:
// create a new bundle by extending the Bundle
class Bundle1 extends Bundle {
  val a = UInt(8.W)
}

// create another bundle by extending the bundle
class Bundle2 extends Bundle1 {
  val b = UInt(16.W)
}

class BadTypeModule extends Module {
  val io = IO(new Bundle {

    // create the clock
    val c  = Input(Clock())

    // create the input of 2bits
    val in = Input(UInt(2.W))

    // create one bit output
    val out = Output(Bool())

    // create an Input of Bundle2
    val bundleIn = Input(new Bundle2)

    // create an Output of Bundle1
    val bundleOut = Output(new Bundle1)
  })
  
  // connecting a clock to a boolean is not valid!
  // io.out := io.c // won't work due to different types

  // Okay, but Chisel will truncate the input width to 1 to match the output.
  // io.out := io.in

  // Compiles; Chisel will connect the common subelements of the two Bundles (in this case, 'a').
  // but there is an error for some reason! maybe due to some version changes
  // io.bundleOut := io.bundleIn
}

println(getVerilog(new BadTypeModule))

Elaborating design...
[error] 	at ... ()
[error] 	at ammonite.$sess.cmd19$Helper$BadTypeModule$$anon$1.<init>(cmd19.sc:23)
[error] 	at ammonite.$sess.cmd19$Helper$BadTypeModule.<init>(cmd19.sc:11)
[error] 	at ammonite.$sess.cmd19$Helper.$anonfun$res19_3$1(cmd19.sc:40)
[error] 	at ... ()
[error] 	at ... (Stack trace trimmed to user code only. Rerun with --full-stacktrace to see the full stack trace)


: 

---
# Type Generics<a name="type-generics"></a>
Scala's generic types (also known as polymorphism) is very complicated, especially when coupling it with inheritance.

This section will just get your toes wet; to understand more, check out [this tutorial](https://twitter.github.io/scala_school/type-basics.html).

Classes can be polymorphic in their types. One good example is sequences, which require knowing their contained type.

In [ ]:
val seq1 = Seq("1", "2", "3") // Type is Seq[String]
val seq2 = Seq(1, 2, 3)       // Type is Seq[Int]
val seq3 = Seq(1, "2", true)  // Type is Seq[Any]

seq1: Seq[String] = List("1", "2", "3")
seq2: Seq[Int] = List(1, 2, 3)
seq3: Seq[Any] = List(1, "2", true)

Sometimes, the Scala compiler needs help determining a polymorphic type, which requires the user to explicitly put the type:

In [ ]:
//val default = Seq() // Error!

// creates an empty sequence of strings.
// this type hint is essential — otherwise Scala would not know what type the result of foldLeft should be
val default = Seq[String]() // User must tell compiler that default is of type Seq[String]

// Folds the sequence from left to right. Starts with default (Seq[String]()) as the accumulator
Seq(1, "2", true).foldLeft(default){ (strings, next) =>
    // strings: the accumulator (starts as Seq[String](), and grows over time).
    // next: the current element from the original sequence (e.g., 1, then "2", then true)
    next match {

        // if the current item is a string, append that to the list
        case s: String => strings ++ Seq(s)

        // otherwise just return the string sequence
        case _ => strings
    }
}

default: Seq[String] = List()
res24_1: Seq[String] = List("2")

Functions can also be polymorphic in their input or output types. The following example defines a function that times how long it takes to run a block of code. It is parameterized based on the return type of the block of code. *Note that the `=> T` syntax encodes an anonymous function that does not have an argument list, e.g. `{ ... }` versus `{ x => ... }`.*

In [ ]:
// [T] : This is a type parameter — it means the function is generic and can work with any return type T, value of the T is based on the code block
// This is a parameter named block, which is a call-by-name expression returning type T.
// This means the function time will return a value of type T, whatever that may be.
// This is powerful because we can execute any code block inside this function
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

// Adds 1 through a million
val int = time { (1 to 1000000).reduce(_ + _) }
println(s"Add 1 through a million is $int")

// Finds the largest number under a million that, in hex, contains "beef"
val string = time {
    (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
}
println(s"The largest number under a million that has beef: $string")

Block took 16.9167 milliseconds!
Add 1 through a million is 1784293664
Block took 88.667 milliseconds!
The largest number under a million that has beef: ebeef


defined function time
int: Int = 1784293664
string: String = "ebeef"

## Chisel Type Hierarchy
To write type generic code with Chisel, it is helpful to know a bit about the type hierarchy of Chisel.

`chisel3.Data` is the base class for Chisel hardware types.
`UInt`, `SInt`, `Vec`, `Bundle`, etc. are all instances of `Data`.
`Data` can be used in IOs and supports `:=`, wires, regs, etc.

Registers are a good example of polymorphic code in Chisel.
Look at the implementation of `RegEnable` (a register with a `Bool` enable signal) [here](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10).
The apply function is templated for `[T <: Data]`, which means `RegEnable` will work for all Chisel hardware types.

Some operations are only defined on subtypes of `Bits`, for example `+`.
This is why you can add `UInt`s or `SInt`s but not `Bundle`s or `Vec`s.

<span style="color:blue">**Example: Type Generic ShiftRegister**<a name="type-generic-shift-register"></a></span><br>
In Scala, objects and functions aren't the only things we can treat as parameters.
We can also treat types as parameters.

We usually need to provide a type constraint.
In this case, we want to be able to put objects in a bundle, connect (:=) them, and create registers with them (RegNext).
These operations cannot be done on arbitrary objects; for example wire := 3 is illegal because 3 is a Scala Int, not a Chisel UInt.
If we use a type constraint to say that type T is a subclass of Data, then we can use := on any objects of type T because := is defined for all Data.

Here is an implementation of a shift register that take types as a parameter.
*gen* is an argument of type T that tells what width to use, for example new ShiftRegister(UInt(4.W)) is a shift register for 4-bit UInts.
*gen* also allows the Scala compiler to infer the type T- you can write new ShiftRegister[UInt](UInt(4.W)) if you want to to be more specific, but the Scala compiler is smart enough to figure it out if you leave out the [UInt].

In [ ]:
// T <: chisel3.Data  means T is a chisel hardware type
// gen is of type T, here T is used to pass the chisel hardware type i.e. we are passing the type as an argument
// n is of type n, which is number of delays or shifts
class ShiftRegisterIO[T <: chisel3.Data](gen: T, n: Int) extends Bundle {

    // number of shifts is not negative
    require (n >= 0, "Shift register must have non-negative shift")
    
    // gen can be something like UInt(4.W)
    val in = Input(gen)

    // output is a vector which has n+1 values with gen (chisel hardware types)
    // we are outputting the io.in as well that is why we do + 1 to n
    val out = Output(Vec(n + 1, gen)) // + 1 because in is included in out

    // Required in some older versions of Chisel to support proper type cloning at runtime.
    override def cloneType: this.type = (new ShiftRegisterIO(gen, n)).asInstanceOf[this.type]
}

class ShiftRegister[T <: chisel3.Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

// visualize(() => new ShiftRegister(SInt(6.W), 3)) // image is generated
test(new ShiftRegister(SInt(6.W), 3)) { c => 
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        c.io.in.poke(i.S) // magic literal creation
        println(s"$i: ${c.io.out.indices.map { index => c.io.out(index).peek().litValue} }")
        c.clock.step(1)
    }}

Elaborating design...
Done elaborating.
Testing ShiftRegister of type SInt<6>(IO io_in in ShiftRegister) and depth 4
0: Vector(0, 0, 0, 0)
1: Vector(1, 0, 0, 0)
2: Vector(2, 1, 0, 0)
3: Vector(3, 2, 1, 0)
4: Vector(4, 3, 2, 1)
5: Vector(5, 4, 3, 2)
6: Vector(6, 5, 4, 3)
7: Vector(7, 6, 5, 4)
8: Vector(8, 7, 6, 5)
9: Vector(9, 8, 7, 6)
test ShiftRegister Success: 0 tests passed in 12 cycles in 0.010699 seconds 1121.65 Hz


defined class ShiftRegisterIO
defined class ShiftRegister

We generally recommend avoiding to use inheritance with type generics.
It can be very tricky to do properly and can get frustrating quickly.

## Type Generics with Typeclasses

The example above was limited to simple operations that could be performed on any instance of `Data` such as `:=` or `RegNext()`.
When generating DSP circuits, we would like to do mathematical operations like addition and multiplication.
The `dsptools` library provides tools for writing type parameterized DSP generators.

Here is an example of writing a multiply-accumulate module.
It can be used to generate a multiply-accumulate (MAC) for `FixedPoint`, `SInt`, or even `DspComplex[T]` (the complex number type provided by `dsptools`).
The syntax of the type bound is a little different because `dsptools` uses typeclasses.
They are beyond the scope of this notebook.
Read the `dsptools` readme and documentation for more information on using typeclasses.

`T <: Data : Ring` means that `T` is a subtype of `Data` and is also a `Ring` .
`Ring` is defined in `dsptools` as a number with `+` and `*` (among other operations).

_An alternative to `Ring` would be `Real`, but that would not allow us to make a MAC for `DspComplex()` because complex numbers are not `Real`._



In [ ]:
import chisel3.experimental._
import dsptools.numbers._

class Mac[T <: chisel3.Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn)
        val b = Input(genIn)
        val c = Input(genIn)
        val out = Output(genOut)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(UInt(4.W), UInt(6.W)) ))
println(getVerilog(new Mac(SInt(4.W), SInt(6.W)) ))
println(getVerilog(new Mac(FixedPoint(4.W, 3.BP), FixedPoint(6.W, 4.BP))))


Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a,
  input  [3:0] io_b,
  input  [3:0] io_c,
  output [5:0] io_out
);
  wire [7:0] _T = io_a * io_b; // @[UIntTypeClass.scala 40:41]
  wire [7:0] _GEN_0 = {{4'd0}, io_c}; // @[UIntTypeClass.scala 19:40]
  wire [7:0] _T_2 = _T + _GEN_0; // @[UIntTypeClass.scala 19:40]
  assign io_out = _T_2[5:0]; // @[cmd18.sc 11:12]
endmodule

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a,
  input  [3:0] io_b,
  input  [3:0] io_c,
  output [5:0] io_out
);
  wire [7:0] _T = $signed(io_a) * $signed(io_b); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _GEN_0 = {{4{io_c[3]}},io_c}; // @[SIntTypeClass.scala 19:40]
  wire [7:0] _T_3 = $signed(_T) + $signed(_GEN_0); // @[SIntTypeClass.scala 19:40]
  assign io_out = _T_3[5:0]; // @[cmd18.sc 11:12]
endmodule

Elaborating design...
Done elaborating.
module Mac(
  input        clock,

import chisel3.experimental._

import dsptools.numbers._


defined class Mac

<span style="color:red">**Exercise: Mac as Object**</span><br>

The Mac `Module` has a small number of inputs and just one output.
It might be convenient for other Chisel generators to write code like
```scala
val out = Mac(a, b, c)
```

Implement an `apply` method in the `Mac` companion object below that implements the `Mac` functionality.

In [ ]:
import dsptools.numbers._

object Mac {
    def apply[T <: chisel3.Data : Ring](a: T, b: T, c: T): T = {
        a * b + c
    }
}

class MacTestModule extends Module {
    val io = IO(new Bundle {
        val uin = Input(UInt(4.W))
        val uout = Output(UInt())
        val sin = Input(SInt(4.W))
        val sout = Output(SInt())
        //val fin = Input(FixedPoint(16.W, 12.BP))
        //val fout = Output(FixedPoint())
    })
    // for each IO pair, do out = in * in + in

    // no new keyword, which means this uses companion object
    // do the operation of a * b + c then it will get assigned to io.uout
    io.uout := Mac(io.uin, io.uin, io.uin)
    io.sout := Mac(io.sin, io.sin, io.sin)
    //io.fout := Mac(io.fin, io.fin, io.fin)
}
println(getVerilog(new MacTestModule))

Elaborating design...
Done elaborating.
module MacTestModule(
  input        clock,
  input        reset,
  input  [3:0] io_uin,
  output [7:0] io_uout,
  input  [3:0] io_sin,
  output [7:0] io_sout
);
  wire [7:0] _T = io_uin * io_uin; // @[UIntTypeClass.scala 40:41]
  wire [7:0] _GEN_0 = {{4'd0}, io_uin}; // @[UIntTypeClass.scala 19:40]
  wire [7:0] _T_3 = $signed(io_sin) * $signed(io_sin); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _GEN_1 = {{4{io_sin[3]}},io_sin}; // @[SIntTypeClass.scala 19:40]
  assign io_uout = _T + _GEN_0; // @[UIntTypeClass.scala 19:40]
  assign io_sout = $signed(_T_3) + $signed(_GEN_1); // @[SIntTypeClass.scala 19:40]
endmodule



import dsptools.numbers._


defined object Mac
defined class MacTestModule

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

        a * b + c

</pre></article></div></section></div>

<span style="color:red">**Exercise: Integrator**</span><br>
Implement an integrator as pictured below. $n_1$ is the width of `genReg` and $n_2$ is the width of `genIn`.

Don't forget that `Reg`, `RegInit`, `RegNext`, `RegEnable`, etc. are templated for types `T <: Data`.

<img src="images/integrator.svg" alt="Integrator" style="width: 250px;"/>

In [ ]:
import dsptools.numbers._

class Integrator[T <: chisel3.Data : Ring](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn)
        val out = Output(genReg)
    })
    
    // My answer
    // val reg = RegNext(io.in + io.out)
    // io.out := reg

    // a more accurate one
    val reg = RegInit(genReg,Ring[T].zero)
    reg := reg + io.in
    io.out := reg
}

test(new Integrator(SInt(4.W), SInt(8.W))) { c =>
    c.io.in.poke(3.S)
    c.io.out.expect(0.S)
    c.clock.step(1)
    c.io.in.poke(-4.S)
    c.io.out.expect(3.S)
    c.clock.step(1)
    c.io.in.poke(6.S)
    c.io.out.expect(-1.S)
    c.clock.step(1)
    c.io.out.expect(5.S)
}

Elaborating design...
Done elaborating.
test Integrator Success: 0 tests passed in 5 cycles in 0.006224 seconds 803.41 Hz


import dsptools.numbers._


defined class Integrator

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

class Integrator\[T <: Data : Ring\](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn.cloneType)
        val out = Output(genReg.cloneType)
    })
    
    val reg = RegInit(genReg, Ring[T].zero) // init to zero
    reg := reg + io.in
    io.out := reg
}

</pre></article></div></section></div>

---
# Creating a Custom Type<a name="creating-a-custom-type"></a>

One of the things that makes Chisel powerful is its extensibility.
You can add your own types that have their own operations and representations that are tailored to your application.
This section will introduce ways to make custom types.

<span style="color:blue">**Example: DspComplex**</span><br>
`DspComplex` is a custom data type defined in **dsptools** [here](https://github.com/ucb-bar/dsptools/blob/v1.0.0/src/main/scala/dsptools/numbers/chisel_concrete/DspComplex.scala#L59).
The key line to understand is this:
```scala
class DspComplex[T <: Data:Ring](val real: T, val imag: T) extends Bundle { ... }
```

`DspComplex` is a type-generic container.
That means the real and imaginary parts of a complex number can be any type as long as they satisfy the type constraints, given by `T <: Data : Ring`.

`T <: Data` means `T` is a subtype of `chisel3.Data`, the base type for Chisel objects.
This means that `DspComplex` only works for objects that are Chisel types and not arbitrary Scala types.

`T : Ring` means that a Ring typeclass implementation for `T` exists.
`Ring` typeclasses define `+` and `*` operators as well as additive and multiplicative identities (see [this Wikipedia article](https://en.wikipedia.org/wiki/Ring_(mathematics)) for details about rings).
**dsptools** defines typeclasses for commonly used Chisel types [here](https://github.com/ucb-bar/dsptools/tree/v1.0.0/src/main/scala/dsptools/numbers/chisel_types).

**dsptools** also defines a `Ring` typeclass for `DspComplex`, so we can reuse our MAC generator with complex numbers:

In [11]:
import dsptools.numbers._
import chisel3.experimental._

class Mac[T <: chisel3.Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn)
        val b = Input(genIn)
        val c = Input(genIn)
        val out = Output(genOut)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(DspComplex(SInt(4.W), SInt(4.W)), DspComplex(SInt(6.W), SInt(6.W))) ))

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a_real,
  input  [3:0] io_a_imag,
  input  [3:0] io_b_real,
  input  [3:0] io_b_imag,
  input  [3:0] io_c_real,
  input  [3:0] io_c_imag,
  output [5:0] io_out_real,
  output [5:0] io_out_imag
);
  wire [3:0] _T_2 = $signed(io_b_real) + $signed(io_b_imag); // @[SIntTypeClass.scala 19:40]
  wire [3:0] _T_5 = $signed(io_a_real) + $signed(io_a_imag); // @[SIntTypeClass.scala 19:40]
  wire [3:0] _T_8 = $signed(io_a_imag) - $signed(io_a_real); // @[SIntTypeClass.scala 29:50]
  wire [7:0] _T_9 = $signed(io_a_real) * $signed(_T_2); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_10 = $signed(_T_5) * $signed(io_b_imag); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_11 = $signed(_T_8) * $signed(io_b_real); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_14 = $signed(_T_9) - $signed(_T_10); // @[SIntTypeClass.scala 29:50]
  wire [7:0] _T_17 = $signed(_T_9) + $signed(_T_11); // @[SI

import dsptools.numbers._

import chisel3.experimental._


defined class Mac

<span style="color:red">**Exercise: Sign-magnitude Numbers**</span><br>
Suppose you wanted to use a sign-magnitude representation and want to reuse all of your DSP generators.
Typeclasses enable this kind of ad-hoc polymorphism.
The following example gives the beggining of an implementation of a SignMagnitude type as well as an implementation of a `Ring` typeclass that will allow the type to be used with the Mac generator.

Fill in implementations for `+` and `*`.
You should pattern them after the implementation for `unary_-()`.
The next block contains a test that checks the correctness of a `Mac` that uses `SignMagnitude`.

In [18]:
import dsptools.numbers._
// SignMagnitude is a class which extends Bundle which is used to define io
// MagnitudeWidth is for representing the number of bits for the width of the value bits
// This is an option, defualt value would be None
class SignMagnitude(val magnitudeWidth: Option[Int] = None) extends Bundle {

    // sign is a boolean value, 0 for positive , -1 for negative
    val sign = Bool()

    // magnitude is based on the passed parameter
    val magnitude = magnitudeWidth match {

        // if the passed parameter is a Some(w) , we use w to create the width
        case Some(w) => UInt(w.W)

        // if its None, then create the UInt without any ints
        case None    => UInt()
    }


    def +(that: SignMagnitude): SignMagnitude = {
        // Implement this!
        val result = Wire(new SignMagnitude())
        when(this.sign===that.sign){
            result.sign := that.sign
            result.magnitude := this.magnitude + that.magnitude
        }.otherwise{
            when(that.magnitude>this.magnitude){
                result.sign := that.sign
                result.magnitude := that.magnitude - this.magnitude
            }.otherwise{
                result.sign := this.sign
                result.magnitude := this.magnitude - that.magnitude
            }
        }
        result
    }

    def -(that: SignMagnitude): SignMagnitude = {
        this.+(-that)
    }
    def unary_-(): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := !this.sign
        result.magnitude := this.magnitude
        result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        // Implement this!
        val result = Wire(new SignMagnitude())
        when(this.sign===that.sign){ // both are positive or both are negative
            result.sign := this.sign
        }.otherwise{    // both have different signs
            result.sign := true.B
        }
        result.magnitude := this.magnitude * that.magnitude
        result
    }
}
trait SignMagnitudeRing extends Ring[SignMagnitude] {
    def plus(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f + g
    }
    def times(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f * g
    }
    def one: SignMagnitude = {
        val one = Wire(new SignMagnitude(Some(1)))
        one.sign := false.B
        one.magnitude := 1.U
        one
    }
    def zero: SignMagnitude = {
        val zero = Wire(new SignMagnitude(Some(0)))
        zero.sign := false.B
        zero.magnitude := 0.U
        zero
    }
    def negate(f: SignMagnitude): SignMagnitude = {
        -f
    }
    
    // Leave unimplemented for this example
    def minusContext(f: SignMagnitude, g: SignMagnitude): SignMagnitude = ???
    def negateContext(f: SignMagnitude): SignMagnitude = ???
    def plusContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
    def timesContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
}
implicit object SignMagnitudeRingImpl extends SignMagnitudeRing

import dsptools.numbers._
// SignMagnitude is a class which extends Bundle which is used to define io
// MagnitudeWidth is for representing the number of bits for the width of the value bits
// This is an option, defualt value would be None

defined class SignMagnitude
defined trait SignMagnitudeRing
defined object SignMagnitudeRingImpl

In [19]:
import chisel3.experimental.BundleLiterals._

test(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) { c =>
    c.io.a.poke(chiselTypeOf(c.io.a).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.b.poke(chiselTypeOf(c.io.b).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.c.poke(chiselTypeOf(c.io.c).Lit(_.sign -> false.B, _.magnitude -> 2.U))
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 11.U))

    c.io.c.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 7.U))

    c.io.b.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> true.B, _.magnitude -> 11.U))
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test Mac Success: 0 tests passed in 2 cycles in 0.020668 seconds 96.77 Hz
SUCCESS!!


import chisel3.experimental.BundleLiterals._



Look at the verilog to see if the output looks reasonable:

In [20]:
println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input        io_a_sign,
  input  [3:0] io_a_magnitude,
  input        io_b_sign,
  input  [3:0] io_b_magnitude,
  input        io_c_sign,
  input  [3:0] io_c_magnitude,
  output       io_out_sign,
  output [4:0] io_out_magnitude
);
  wire  _GEN_0 = io_a_sign == io_b_sign ? io_a_sign : 1'h1; // @[cmd17.sc 51:36 cmd17.sc 52:25 cmd17.sc 54:25]
  wire [7:0] _T_1 = io_a_magnitude * io_b_magnitude; // @[cmd17.sc 56:44]
  wire [7:0] _GEN_5 = {{4'd0}, io_c_magnitude}; // @[cmd17.sc 26:48]
  wire [7:0] _T_4 = _T_1 + _GEN_5; // @[cmd17.sc 26:48]
  wire [7:0] _T_7 = _GEN_5 - _T_1; // @[cmd17.sc 30:52]
  wire [7:0] _T_9 = _T_1 - _GEN_5; // @[cmd17.sc 33:52]
  wire  _GEN_1 = _GEN_5 > _T_1 ? io_c_sign : _GEN_0; // @[cmd17.sc 28:48 cmd17.sc 29:29 cmd17.sc 32:29]
  wire [7:0] _GEN_2 = _GEN_5 > _T_1 ? _T_7 : _T_9; // @[cmd17.sc 28:48 cmd17.sc 30:34 cmd17.sc 33:34]
  wire [7:0] _GEN_4 = _GEN_0 == io_c_sign 

`SignMagnitude` even works with `DspComplex`!

In [21]:
println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input        io_a_real_sign,
  input  [3:0] io_a_real_magnitude,
  input        io_a_imag_sign,
  input  [3:0] io_a_imag_magnitude,
  input        io_b_real_sign,
  input  [3:0] io_b_real_magnitude,
  input        io_b_imag_sign,
  input  [3:0] io_b_imag_magnitude,
  input        io_c_real_sign,
  input  [3:0] io_c_real_magnitude,
  input        io_c_imag_sign,
  input  [3:0] io_c_imag_magnitude,
  output       io_out_real_sign,
  output [4:0] io_out_real_magnitude,
  output       io_out_imag_sign,
  output [4:0] io_out_imag_magnitude
);
  wire [3:0] _T_2 = io_b_real_magnitude + io_b_imag_magnitude; // @[cmd17.sc 26:48]
  wire [3:0] _T_5 = io_b_imag_magnitude - io_b_real_magnitude; // @[cmd17.sc 30:52]
  wire [3:0] _T_7 = io_b_real_magnitude - io_b_imag_magnitude; // @[cmd17.sc 33:52]
  wire  _GEN_0 = io_b_imag_magnitude > io_b_real_magnitude ? io_b_imag_sign : io_b_real_sign; // @[cmd17.s

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">
    // implementations for class SignMagnitude

    def +(that: SignMagnitude): SignMagnitude = {
      val result = Wire(new SignMagnitude())
      val signsTheSame = this.sign === that.sign
      when (signsTheSame) {
        result.sign      := this.sign
        result.magnitude := this.magnitude + that.magnitude
      } .otherwise {
        when (this.magnitude > that.magnitude) {
          result.sign      := this.sign
          result.magnitude := this.magnitude - that.magnitude
        } .otherwise {
          result.sign      := that.sign
          result.magnitude := that.magnitude - this.magnitude
        }   
      }   
      result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := this.sign ^ that.sign
        result.magnitude := this.magnitude * that.magnitude
        result
    }


</pre></article></div></section></div>